In [3]:
import charts
import pymongo

Server running in the folder C:\Users\ZhangLi at 127.0.0.1:53876


In [4]:
# 数据库连接使用，每次都需要运行
client = pymongo.MongoClient('localhost',27017)
ceshi = client['ceshi']
information = ceshi['sample_info_1']

In [12]:
# 测试
for i in information.find().limit(1):
    print(i)

{'pub_date': '2015.12.27', '_id': ObjectId('5698f525a98063dbe7e91ca8'), 'url': 'http://bj.58.com/diannao/24475337853109x.shtml', 'price': '1500', 'look': '-', 'title': '【图】三星 A5 白色 没有打开过 - 朝阳望京台式机/配件 - 北京58同城', 'cates': '北京二手台式机', 'area': ['朝阳', '-', '望京']}


In [33]:
areas_list = []  # 去具体的地区中的第一个
for i in information.find():  # 发现，地区信息中为list和none还有一种空的list，所以在list的类型中计算了长度
    if i['area'] == None:
        area = '不明'     
    else :
        length = len(i['area'])  
        if length == 0:
            area = '无语'
        else:
            area = i['area'][0]
    information.update_one({'_id':i['_id']},{'$set':{'area':area}}) # 更新到数据库中
    areas_list.append(area)
area_list =list(set(areas_list))
print(area_list,type(area_list))

['海淀', '崇文', '大兴', '东城', '宣武', '石景山', '门头沟', '北京周边', '西城', '附近', '顺义', '通州', '怀柔', '平谷', '密云', '燕郊', '昌平', '丰台', '延庆', '无语', '朝阳', '不明', '房山'] <class 'list'>


In [10]:
cates_list = [] # 筛选出所有的大类目,并更新到数据库
for i in information.find():  # 如果说使用find方法的话，怎么使用比较好呢？
    cate = i['cates'][2].split('/')[0]
    information.update_one({'_id':i['_id']},{'$set':{'cates':cate}})
    cates_list.append(cate)
cate_list = list(set(cates_list))
print(cate_list)

['北京二手图书', '北京二手设备', '北京二手平板电脑', '北京二手母婴', '北京二手家电', '北京二手服装', '北京二手美容', '北京二手文体', '北京二手办公用品', '北京二手笔记本', '北京二手家具', '北京二手数码产品', '北京二手手机', '北京其他二手物品', '北京二手台式机']


In [102]:
def data_gen():  # series的数据生成器
    a = []
    for cate in cate_list:
        for i in information.find({'pub_date':{'$in':['2016-01-12','2016-01-14']}},{'area':'海淀','cates':cate}):  # 这个的问题出在哪里了
            a.append(i)
        data  = {
            'name':cate,
            'data':[len(a)],
            'type':'column'
        }
        yield data

In [116]:
series = [i for i in data_gen()] # 画图标

charts.plot(series, show='inline')

In [16]:
for i in information.find(): # 统一时间的格式为2.2.2
    a = i['pub_date'].split('-')
    if len(a) > 1:
        date = '{}.{}.{}'.format(a[0],a[1],a[2])
    else:
        date = a[0]
    information.update_one({'_id':i['_id']},{'$set':{'pub_date':date}})

In [5]:
cates_list = [] # 上边程序程序中已经筛选出所有的大类目，现在读出所有的数据
for i in information.find():  # 如果说使用find方法的话，怎么使用比较好呢？
    cate = i['cates']
    cates_list.append(cate)
cate_list = list(set(cates_list))
print(cate_list)

['北京二手办公用品', '北京二手文体', '北京二手美容', '北京二手家具', '北京二手母婴', '北京二手设备', '北京二手台式机', '北京二手笔记本', '北京二手数码产品', '北京二手服装', '北京二手家电', '北京二手图书', '北京二手手机', '北京其他二手物品', '北京二手平板电脑']


In [21]:
def price_gen(start_data, end_data, cate_list,types):  # series数据生成函数，但是一直在报错
    for cate in cate_list:
        count = 0
        num = 0 
        for i in information.find({'pub_date':{'$in':['start_data','end_data']},'cates':cate}):
            a = list(filter(str.isdigit,i['price']))
            if len(a) == 0:
                count = count
            else:
                string = "".join(a)
                int_str = int(string)
                count = count + int_str
                num += 1
        average = count/num
        data = {
            'name':cate,
            'data':[average],
            'type':types
        }
        yield data

In [28]:
for data in price_gen('2016.01.10','2016.01.20',cate_list,'line'): # 测试使用，一直在报错
    print(data) 

ZeroDivisionError: division by zero

In [32]:
# series = [data for data in price_gen('2016.01.10','2016.01.20',cate_list,'line')]
series = price_list
options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': '各大类目对应的平均价格'},
    'subtitle': {'text': 'Source: haha.com'},
    'xAxis'   : {'categories': [cate for cate in cate_list]},
    'yAxis'   : {'title': {'text': 'average price'}}
    }
charts.plot(series, show = 'inline', options = options)

In [31]:
price_list = []  # 数据生成器总是存在问题，所以使用另一种方法生成数据
for cate in cate_list:
    count = 0
    num = 0 
    for i in information.find({'pub_date':{'$in':['2016.01.10','2016.01.20']},'cates':cate}):
        a = list(filter(str.isdigit,i['price']))
        if len(a) == 0:
            count = count
        else:
            string = "".join(a)
            int_str = int(string)
            count = count + int_str
            num += 1
    average = count/num
    data = {
        'name':cate,
        'data':[average],
        'type':'column'
    }
    price_list.append(data)
print(price_list)

[{'name': '北京二手办公用品', 'type': 'column', 'data': [838.0967741935484]}, {'name': '北京二手文体', 'type': 'column', 'data': [2638.7070707070707]}, {'name': '北京二手美容', 'type': 'column', 'data': [783.469387755102]}, {'name': '北京二手家具', 'type': 'column', 'data': [1318.923076923077]}, {'name': '北京二手母婴', 'type': 'column', 'data': [497.2142857142857]}, {'name': '北京二手设备', 'type': 'column', 'data': [40848.621212121216]}, {'name': '北京二手台式机', 'type': 'column', 'data': [853.5911111111111]}, {'name': '北京二手笔记本', 'type': 'column', 'data': [4596.364285714286]}, {'name': '北京二手数码产品', 'type': 'column', 'data': [17199.254945054945]}, {'name': '北京二手服装', 'type': 'column', 'data': [998.8896882494005]}, {'name': '北京二手家电', 'type': 'column', 'data': [5202.178489702517]}, {'name': '北京二手图书', 'type': 'column', 'data': [1182.6823529411765]}, {'name': '北京二手手机', 'type': 'column', 'data': [1561.8241758241759]}, {'name': '北京其他二手物品', 'type': 'column', 'data': [1164.046511627907]}, {'name': '北京二手平板电脑', 'type': 'column', 'data': [1